## Imports

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import pandas as pd
import numpy as np
from raw_data import save_all_files, concat_df

## Load the data

Loads the required datasets from google cloud. Then selects only relevant columns and saves it into /data/_year_.pkl.

In [57]:
save_all_files()

Get a coffee ;) 
You already have this file's corresponding .pkl file. No need to download the .csv file again.
You already have this file's corresponding .pkl file. No need to download the .csv file again.
You already have this file's corresponding .pkl file. No need to download the .csv file again.
You already have this file's corresponding .pkl file. No need to download the .csv file again.
You already have this file's corresponding .pkl file. No need to download the .csv file again.
You successfully downloaded 0 of the 5 files.


## Concatenate data from different years to one dataframe

In [58]:
df = concat_df()
df

Reading file: 2011.pkl
Reading file: 2012.pkl
Reading file: 2013.pkl
Reading file: 2014.pkl
Reading file: 2015.pkl


,DIABETE3,SEX,_AGEG5YR,EDUCA,_BMI5,_BMI5CAT,GENHLTH,PHYSHLTH,_TOTINDA,EXERANY2,...,SMOKDAY2,_RFSMOK3,DRNKANY5,ALCDAY5,AVEDRNK2,DRNK3GE5,_RFBING5,CVDSTRK3,CVDINFR4,CVDCRHD4
0,3.0,2.0,9.0,4.0,1855.0,2.0,4.0,88.0,2.0,2.0,...,1.0,2.0,1.0,103.0,2.0,88.0,1.0,2.0,2.0,2.0
1,3.0,1.0,3.0,3.0,3529.0,4.0,4.0,12.0,2.0,2.0,...,1.0,2.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0
2,3.0,2.0,14.0,6.0,1695.0,1.0,2.0,88.0,1.0,1.0,...,3.0,1.0,1.0,101.0,1.0,88.0,1.0,2.0,2.0,2.0
3,3.0,2.0,14.0,4.0,2579.0,3.0,3.0,88.0,9.0,NaN,...,NaN,1.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0
4,4.0,2.0,7.0,3.0,4346.0,4.0,5.0,25.0,1.0,1.0,...,3.0,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380042,1.0,2.0,11.0,2.0,1842.0,1.0,4.0,88.0,2.0,2.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
2380043,3.0,2.0,2.0,5.0,2834.0,3.0,1.0,88.0,1.0,1.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
2380044,3.0,2.0,11.0,4.0,4110.0,4.0,4.0,88.0,9.0,NaN,...,3.0,1.0,1.0,202.0,2.0,88.0,1.0,2.0,2.0,2.0
2380045,3.0,1.0,7.0,5.0,2315.0,2.0,3.0,88.0,2.0,2.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0


### Adding gestanional diabetes mellitus (gdm) to the dataframe

In [59]:
# aus Target_name Frauen mit Gestationsdiabetes (diabetes during pregnancy) filtern
filt_df_gdm = df["DIABETE3"]==2

# Gestationsdiabetes als neue Spalte ("GES_DIABETES) einfügen
df["GDM"]=filt_df_gdm

# Ersetzen von False zu 0 (keine Gestationsdiabetes) und True zu 1 (Gestationsdiabetes)
df["GDM"].replace([False, True], [0,1], inplace=True)
print(df.GDM.value_counts())

# Verschieben der Gestationsdiabetes-Spalte auf Position 7
df.insert(loc=6, column="GDM", value=df.pop("GDM"))
df

0    2358730
1      21317
Name: GDM, dtype: int64


,DIABETE3,SEX,_AGEG5YR,EDUCA,_BMI5,_BMI5CAT,GDM,GENHLTH,PHYSHLTH,_TOTINDA,...,SMOKDAY2,_RFSMOK3,DRNKANY5,ALCDAY5,AVEDRNK2,DRNK3GE5,_RFBING5,CVDSTRK3,CVDINFR4,CVDCRHD4
0,3.0,2.0,9.0,4.0,1855.0,2.0,0,4.0,88.0,2.0,...,1.0,2.0,1.0,103.0,2.0,88.0,1.0,2.0,2.0,2.0
1,3.0,1.0,3.0,3.0,3529.0,4.0,0,4.0,12.0,2.0,...,1.0,2.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0
2,3.0,2.0,14.0,6.0,1695.0,1.0,0,2.0,88.0,1.0,...,3.0,1.0,1.0,101.0,1.0,88.0,1.0,2.0,2.0,2.0
3,3.0,2.0,14.0,4.0,2579.0,3.0,0,3.0,88.0,9.0,...,NaN,1.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0
4,4.0,2.0,7.0,3.0,4346.0,4.0,0,5.0,25.0,1.0,...,3.0,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380042,1.0,2.0,11.0,2.0,1842.0,1.0,0,4.0,88.0,2.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
2380043,3.0,2.0,2.0,5.0,2834.0,3.0,0,1.0,88.0,1.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0
2380044,3.0,2.0,11.0,4.0,4110.0,4.0,0,4.0,88.0,9.0,...,3.0,1.0,1.0,202.0,2.0,88.0,1.0,2.0,2.0,2.0
2380045,3.0,1.0,7.0,5.0,2315.0,2.0,0,3.0,88.0,2.0,...,NaN,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0


# MODIFY THIS

## Data preparation

- [ ] feature selection
- [ ] data cleaning

In [60]:
# here the data cleaning and data preparation takes place
target_name = "DIABETE3"
feature_names = df.columns.values.tolist()
feature_names.remove(target_name)

# removed all nan values for decision tree to function
# unproblematic features: subset of feature_names with those removed with a lot of nan values

feature_list = [
    "SEX", 
    "_AGEG5YR", 
    "EDUCA", 
    "_BMI5", 
    "_BMI5CAT",
    "GDM",
    "GENHLTH", 
    "PHYSHLTH", 
    "_TOTINDA", 
    "EXERANY2", 
    "SMOKE100", 
    "SMOKDAY2", 
    "_RFSMOK3", 
    "DRNKANY5", 
    "ALCDAY5", 
    "AVEDRNK2", 
    "DRNK3GE5", 
    "_RFBING5", 
    "CVDSTRK3", 
    "CVDINFR4", 
    "CVDCRHD4"
]

renames = [
    "sex",
    "age",
    "education",
    "body_mass_index",
    "bmi_cat",
    "gestational_diabetes",
    "estimated_health",
    "bad_physhealth_days",
    "had_phys_activity",
    "any_exercise",
    "smoked_over_100",
    "smokes_daily",
    "is_smoker",
    "alcohol_past_30",
    "one_drink_per_day_month_last_30",
    "average_drinks",
    "more_than_5_drinks",
    "more_than_5_drinks_on_one_occasion",
    "had_stroke",
    "had_heart_attack",
    "had_angina"
]

# only include rows with diabetes = yes and no and select relevant columns
df_filtered = df.loc[df["DIABETE3"].isin([1, 3]), feature_list + [target_name]]

# rename label 1 = yes (has diabetes) | 3 = no (no diabetes)
df_filtered["DIABETE3"] = df_filtered["DIABETE3"].map({1: "yes", 3: "no"})
df_filtered.columns=renames + [target_name]
df_filtered

,sex,age,education,body_mass_index,bmi_cat,gestational_diabetes,estimated_health,bad_physhealth_days,had_phys_activity,any_exercise,...,is_smoker,alcohol_past_30,one_drink_per_day_month_last_30,average_drinks,more_than_5_drinks,more_than_5_drinks_on_one_occasion,had_stroke,had_heart_attack,had_angina,DIABETE3
0,2.0,9.0,4.0,1855.0,2.0,0,4.0,88.0,2.0,2.0,...,2.0,1.0,103.0,2.0,88.0,1.0,2.0,2.0,2.0,no
1,1.0,3.0,3.0,3529.0,4.0,0,4.0,12.0,2.0,2.0,...,2.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0,no
2,2.0,14.0,6.0,1695.0,1.0,0,2.0,88.0,1.0,1.0,...,1.0,1.0,101.0,1.0,88.0,1.0,2.0,2.0,2.0,no
3,2.0,14.0,4.0,2579.0,3.0,0,3.0,88.0,9.0,NaN,...,1.0,9.0,NaN,NaN,NaN,9.0,2.0,2.0,2.0,no
5,1.0,9.0,4.0,2957.0,3.0,0,2.0,88.0,1.0,1.0,...,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380042,2.0,11.0,2.0,1842.0,1.0,0,4.0,88.0,2.0,2.0,...,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0,yes
2380043,2.0,2.0,5.0,2834.0,3.0,0,1.0,88.0,1.0,1.0,...,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0,no
2380044,2.0,11.0,4.0,4110.0,4.0,0,4.0,88.0,9.0,NaN,...,1.0,1.0,202.0,2.0,88.0,1.0,2.0,2.0,2.0,no
2380045,1.0,7.0,5.0,2315.0,2.0,0,3.0,88.0,2.0,2.0,...,1.0,2.0,888.0,NaN,NaN,1.0,2.0,2.0,2.0,no


### Drop missing and bad input values

In [61]:
nan_amounts = [df_filtered[col].isnull().values.sum() for col in df_filtered.columns]
cols_with_nan_amounts = list(zip(df_filtered.columns, nan_amounts))
cols_with_nan_amounts.sort(key=lambda x: x[1], reverse=True)
cols_with_nan_amounts
# for col in df_filtered.columns:
#     nan_amounts.append()
#     print("\n")
#     print(f"Column {col} has the following values:")
#     print(df_filtered[col].value_counts())
#     print(f"... and {df_filtered[col].isnull().values.sum()} nan values:")
#     print()

[('smokes_daily', 1303910),
 ('more_than_5_drinks', 1203901),
 ('average_drinks', 1202267),
 ('body_mass_index', 141448),
 ('bmi_cat', 141448),
 ('any_exercise', 85401),
 ('one_drink_per_day_month_last_30', 81979),
 ('smoked_over_100', 49303),
 ('education', 3014),
 ('estimated_health', 19),
 ('bad_physhealth_days', 14),
 ('had_heart_attack', 3),
 ('sex', 0),
 ('age', 0),
 ('gestational_diabetes', 0),
 ('had_phys_activity', 0),
 ('is_smoker', 0),
 ('alcohol_past_30', 0),
 ('more_than_5_drinks_on_one_occasion', 0),
 ('had_stroke', 0),
 ('had_angina', 0),
 ('DIABETE3', 0)]

The first 8 cols have a lot of nan-values:
```
('SMOKDAY2', 1303910),
('DRNK3GE5', 1203901),
('AVEDRNK2', 1202267),
('_BMI5', 141448),
('_BMI5CAT', 141448),
('EXERANY2', 85401),
('ALCDAY5', 81979),
('SMOKE100', 49303)
```
Dropping all rows with nan-values drops around 79% of all rows, see below.

In [62]:
# drop na drops around 79% of the rows...
df_prepared = df_filtered.dropna().reset_index(drop=True)
df_prepared.DIABETE3.value_counts()

no     463759
yes     49064
Name: DIABETE3, dtype: int64

# DON'T TOUCH FROM HERE

## Model training
- [ ] model selection
- [ ] train-test split (maybe before data prep)
- [ ] hyperparameter tuning
- [ ] more stuff

In [63]:
X_train, X_test, y_train, y_test = train_test_split(df_prepared[feature_list], df_prepared[target_name], test_size=0.33, random_state=41)

model = DecisionTreeClassifier(min_samples_leaf=10)
model = model.fit(X_train, y_train)

KeyError: "None of [Index(['SEX', '_AGEG5YR', 'EDUCA', '_BMI5', '_BMI5CAT', 'GDM', 'GENHLTH',\n       'PHYSHLTH', '_TOTINDA', 'EXERANY2', 'SMOKE100', 'SMOKDAY2', '_RFSMOK3',\n       'DRNKANY5', 'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', '_RFBING5', 'CVDSTRK3',\n       'CVDINFR4', 'CVDCRHD4'],\n      dtype='object')] are in the [columns]"

In [ ]:
y_predict = model.predict(X_test)

score = accuracy_score(y_test, y_predict)
print(score)

matrix = confusion_matrix(y_test, y_predict)
print(matrix)

precision, recall, fbeta_score, _ = precision_recall_fscore_support(y_true=y_test, y_pred=y_predict, labels=["yes", "no"])

print(f"Precision for 'yes' = {precision[0]}")
print(f"Precision for 'no' = {precision[1]}")
print(f"recall for 'yes' = {recall[0]}")
print(f"recall for 'no' = {recall[1]}")
print(f"fbeta_score for 'yes' = {fbeta_score[0]}")
print(f"fbeta_score for 'no' = {fbeta_score[1]}")

## Use Model to predict user's diabetes probability

### Get user input

In [ ]:
# please input data: ...

# use data for diabetes proba
user_row = df_prepared.loc[[11], feature_list].reset_index(drop=True) # got a random index just as an example
probas = model.predict_proba(user_row)
print(user_row)
# label_meanings = (
#     "diabetes",
#     "diabetes during pregnancy",
#     "pre-diabetes or borderline diabetes",
#     "don't know",
#     "refused",
#     "BLANK"
# )
probas
# for l, p in probas.tolist():
#     print(f"With a probability of {round(p*100, 2)}% you (will) have result: {l}.")

### Make recommendations: What factors have the most significant impact on the user's diabetes probability?